## Testing Julia for GAIA

using the julia language for GAIA analaysis

In [1]:
using PyCall
using DataFrames
using Clustering

@pyimport astropy.io.votable as votable
@pyimport matplotlib.pyplot as plt

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)


In [2]:
## Types

mutable struct Df
    ndata::Int32
    data::Array{Float64}
end


In [3]:
## function to create the df and filter the data
#########
function readVotable(voname::String)
    
    vot = votable.parse(voname)
    data = vot[:get_first_table]() 
    
    println("## Votable $voname  read")
    
    return(data["array"]["data"])
end

#########
function filterData(gaia, dist_range = [0., 2000], vra_range = [-250,250], vdec_range = [-250.,250], mag_range =[-1e9, 1e9])

    ngaia = length(gaia)
    
    lgal = zeros(ngaia)
    bgal = zeros(ngaia)    
    distance = zeros(ngaia)    
    vra = zeros(ngaia)
    vdec = zeros(ngaia)    
    g = zeros(ngaia)
    rp = zeros(ngaia)     
    bp = zeros(ngaia) 
    
    for i in 1:ngaia
        lgal[i]     = convert(Float64,gaia[i]["l"])
        bgal[i]     = convert(Float64,gaia[i]["b"])
        distance[i] = 1000. / convert(Float64,gaia[i]["parallax"])
        pmra  = convert(Float64,gaia[i]["pmra"])
        pmdec = convert(Float64,gaia[i]["pmdec"])
        vra[i]      = 4.74e-3 * pmra  * distance[i]
        vdec[i]     = 4.74e-3 * pmdec * distance[i]
        
        g[i]        = convert(Float64,gaia[i]["phot_g_mean_mag"])
        rp[i]       = convert(Float64,gaia[i]["phot_rp_mean_mag"])
        bp[i]       = convert(Float64,gaia[i]["phot_bp_mean_mag"])
        
        
    end
    
    ## Filtering ...
    i1 =  distance .> dist_range[1]
    i2 =  distance .< dist_range[2]
    i3 =  vra .> vra_range[1]
    i4 =  vra .< vra_range[2]
    i5 =  vdec .> vdec_range[1]
    i6 =  vdec .< vdec_range[2]
    i7 = g  .> mag_range[1]
    i8 = g  .< mag_range[2]
    i9  = rp  .> mag_range[1]
    i10 = rp  .< mag_range[2]
    i11  = bp  .> mag_range[1]
    i12  = bp  .< mag_range[2]
     
    ifinal = i1 .& i2 .& i3 .& i4 .& i5 .& i6 .& i7 .& i8 .& i9 .& i10 .& i11 .& i12 
    
    ## G magnitude
     # gbar =  g[ifinal] - 5. * log10.(distance[ifinal]) + 17.
    gbar =  g[ifinal] - 5.*log10.(distance[ifinal]) .+ 17.
    
    ## Df of the filtered dat
    ndata = length(distance[ifinal])
    s = Df(ndata, zeros(8,ndata))
    
    s.data[1,:] = lgal[ifinal]
    s.data[2,:] = bgal[ifinal]
    s.data[3,:] = distance[ifinal]
    s.data[4,:] = vra[ifinal]
    s.data[5,:] = vdec[ifinal]
    s.data[6,:] = gbar
    s.data[7,:] = g[ifinal] .- rp[ifinal]
    s.data[8,:] = bp[ifinal] .- g[ifinal]
    
    println("## Filtering done ...")
    println("## Stars selected: $ndata")
    
    return(s)
    
end



In [4]:
## Main program
voname = "NGC 2516-1.0deg.vot"

data = readVotable(voname)
source = filterData(data)

println(source.data[:,1])
x = source.data[1,:]
y = source.data[2,:]

# plt.scatter(x, y , s = 0.5 )
# plt.show()

UndefVarError: UndefVarError: readVotable not defined

In [5]:
## Clustering ..
println("## Starting DBSCAN ...")

eps = 2.0 
res = dbscan(source.data , eps , leafsize=20, min_neighbors=1, min_cluster_size=10)
print(res)

## Starting DBSCAN ...


UndefVarError: UndefVarError: source not defined